# Algorithm Benchmark Analysis

- **Classical KEMs:** X25519, P-256, RSA-2048
- **Classical Signatures:** Ed25519, ECDSA-P256, RSA-2048
- **PQC KEMs:** ML-KEM-768, Classic McEliece 460896
- **PQC Signatures:** ML-DSA-65 (Dilithium3)

In [40]:
import pandas as pd
import numpy as np

## 1. Load and EDA

In [ ]:
crypto_df = pd.read_csv('../bench/crypto_algorithms.csv')

print("Dataset shape:", crypto_df.shape)
crypto_df.head(15)

Dataset shape: (510, 7)

First few rows:


,timestamp,algorithm,class,operation,time_ms,ops_per_sec,notes
0,2025-11-09T18:43:17+00:00,X25519,ecdh,ecdh,0.0387,25822.5,classical
1,2025-11-09T18:43:20+00:00,X25519,ecdh,ecdh,0.0337,29664.5,classical
2,2025-11-09T18:43:23+00:00,X25519,ecdh,ecdh,0.0331,30244.8,classical
3,2025-11-09T18:43:26+00:00,X25519,ecdh,ecdh,0.0343,29145.6,classical
4,2025-11-09T18:43:29+00:00,X25519,ecdh,ecdh,0.0319,31386.2,classical
5,2025-11-09T18:43:32+00:00,X25519,ecdh,ecdh,0.0320,31219.0,classical
6,2025-11-09T18:43:35+00:00,X25519,ecdh,ecdh,0.0315,31768.8,classical
7,2025-11-09T18:43:38+00:00,X25519,ecdh,ecdh,0.0317,31571.5,classical
8,2025-11-09T18:43:41+00:00,X25519,ecdh,ecdh,0.0315,31703.0,classical
9,2025-11-09T18:43:44+00:00,X25519,ecdh,ecdh,0.0317,31508.1,classical


In [ ]:
print("Algorithms:", crypto_df['algorithm'].unique())
print("\nOperations:", crypto_df['operation'].unique())
print("\nSamples:")
crypto_df.groupby('algorithm').size()

Unique algorithms: ['X25519' 'P-256' 'RSA-2048' 'Ed25519' 'ECDSA-P256' 'ML-KEM-768'
 'Classic-McEliece-460896' 'ML-DSA-65']

Unique operations: ['ecdh' 'sign' 'verify' 'keygen' 'encap' 'decap']

Samples per algorithm:


algorithm
Classic-McEliece-460896    90
ECDSA-P256                 60
Ed25519                    60
ML-DSA-65                  90
ML-KEM-768                 90
P-256                      30
RSA-2048                   60
X25519                     30
dtype: int64

## 2. Statistics

In [ ]:
stats = crypto_df.groupby(['algorithm', 'class', 'operation'])['time_ms'].agg(['mean', 'std', 'count']).reset_index()

# mean ± std
stats['performance'] = stats.apply(lambda row: f"{row['mean']:.4f} ± {row['std']:.4f}", axis=1)
stats[['algorithm', 'class', 'operation', 'performance', 'count']]

Statistical summary:


,algorithm,class,operation,performance,count
0,Classic-McEliece-460896,kem,decap,0.0991 ± 0.0062,30
1,Classic-McEliece-460896,kem,encap,0.0651 ± 0.0059,30
2,Classic-McEliece-460896,kem,keygen,118.7600 ± 7.0665,30
3,ECDSA-P256,sig,sign,0.0284 ± 0.0011,30
4,ECDSA-P256,sig,verify,0.0631 ± 0.0021,30
5,Ed25519,sig,sign,0.0406 ± 0.0019,30
6,Ed25519,sig,verify,0.1102 ± 0.0071,30
7,ML-DSA-65,sig,keygen,0.0752 ± 0.0034,30
8,ML-DSA-65,sig,sign,0.1434 ± 0.0046,30
9,ML-DSA-65,sig,verify,0.0654 ± 0.0031,30


## 3. Algorithm Performance Table 


In [ ]:
# pivot for mean and std 
pivot_mean = stats.pivot_table(
    index=['algorithm', 'class'], 
    columns='operation', 
    values='mean',
    aggfunc='first'
).reset_index()

pivot_std = stats.pivot_table(
    index=['algorithm', 'class'], 
    columns='operation', 
    values='std',
    aggfunc='first'
).reset_index()

def format_val(mean_val, std_val):
    if pd.isna(mean_val):
        return None
    if pd.isna(std_val):
        return f"{mean_val:.4f}"
    return f"{mean_val:.4f} ± {std_val:.4f}"

# TABLE 2A: KEMs and Key Exchange 
table_2a_mean = pivot_mean[pivot_mean['class'].isin(['kem', 'ecdh'])].copy()
table_2a_std = pivot_std[pivot_std['class'].isin(['kem', 'ecdh'])].copy()

category_map = {'kem': 'KEM', 'ecdh': 'ECDH'}
table_2a_mean['Categoría'] = table_2a_mean['class'].map(category_map)
table_2a_mean['Algoritmo'] = table_2a_mean['algorithm']
table_2a_std['Categoría'] = table_2a_std['class'].map(category_map)
table_2a_std['Algoritmo'] = table_2a_std['algorithm']

# formatted columns
results_2a = []
for idx, row in table_2a_mean.iterrows():
    std_row = table_2a_std[table_2a_std['algorithm'] == row['algorithm']].iloc[0]
    
    result = {
        'Algoritmo': row['Algoritmo'],
        'Categoría': row['Categoría']
    }
    
    if row['Categoría'] == 'ECDH': 
        result['Operación Cliente (ms)'] = format_val(row.get('ecdh'), std_row.get('ecdh'))
        result['Operación Servidor (ms)'] = format_val(row.get('ecdh'), std_row.get('ecdh'))
        
        if pd.notna(row.get('ecdh')):
            total_mean = 2 * row['ecdh']
            total_std = 2 * std_row.get('ecdh', 0)
            result['Tiempo Total (ms)'] = f"{total_mean:.4f} ± {total_std:.4f}"
        else:
            result['Tiempo Total (ms)'] = None
            
    else:  # KEM
        result['Operación Cliente (ms)'] = format_val(row.get('encap'), std_row.get('encap'))
        
        keygen_val = row.get('keygen', 0) if pd.notna(row.get('keygen')) else 0
        decap_val = row.get('decap', 0) if pd.notna(row.get('decap')) else 0
        keygen_std = std_row.get('keygen', 0) if pd.notna(std_row.get('keygen')) else 0
        decap_std = std_row.get('decap', 0) if pd.notna(std_row.get('decap')) else 0
        
        server_mean = keygen_val + decap_val
        server_std = np.sqrt(keygen_std**2 + decap_std**2)
        result['Operación Servidor (ms)'] = f"{server_mean:.4f} ± {server_std:.4f}"
        
        encap_val = row.get('encap', 0) if pd.notna(row.get('encap')) else 0
        encap_std = std_row.get('encap', 0) if pd.notna(std_row.get('encap')) else 0
        
        total_mean = encap_val + keygen_val + decap_val
        total_std = np.sqrt(encap_std**2 + keygen_std**2 + decap_std**2)
        result['Tiempo Total (ms)'] = f"{total_mean:.4f} ± {total_std:.4f}"
    
    results_2a.append(result)

table_2a = pd.DataFrame(results_2a)

# TABLE 2B: Digital Signatures
table_2b_mean = pivot_mean[pivot_mean['class'] == 'sig'].copy()
table_2b_std = pivot_std[pivot_std['class'] == 'sig'].copy()

table_2b_mean['Categoría'] = 'Firma Digital'
table_2b_mean['Algoritmo'] = table_2b_mean['algorithm']
table_2b_std['Categoría'] = 'Firma Digital'
table_2b_std['Algoritmo'] = table_2b_std['algorithm']

results_2b = []
for idx, row in table_2b_mean.iterrows():
    std_row = table_2b_std[table_2b_std['algorithm'] == row['algorithm']].iloc[0]
    
    result = {
        'Algoritmo': row['Algoritmo'],
        'Categoría': row['Categoría']
    }
    
    if pd.notna(row.get('sign')):
        result['Firma (ms)'] = format_val(row['sign'], std_row.get('sign'))
    else:
        result['Firma (ms)'] = None
    
    if pd.notna(row.get('verify')):
        result['Verificación (ms)'] = format_val(row['verify'], std_row.get('verify'))
    else:
        result['Verificación (ms)'] = None
    
    sign_val = row.get('sign', 0) if pd.notna(row.get('sign')) else 0
    verify_val = row.get('verify', 0) if pd.notna(row.get('verify')) else 0
    
    sign_std = std_row.get('sign', 0) if pd.notna(std_row.get('sign')) else 0
    verify_std = std_row.get('verify', 0) if pd.notna(std_row.get('verify')) else 0
    
    total_mean = sign_val + verify_val
    total_std = np.sqrt(sign_std**2 + verify_std**2)
    result['Tiempo Total (ms)'] = f"{total_mean:.4f} ± {total_std:.4f}"
    
    results_2b.append(result)

table_2b = pd.DataFrame(results_2b)

print("\nTABLA 2A")
display(table_2a)

print("\nTABLA 2B")
display(table_2b)


=== TABLA 2A: Rendimiento de Intercambio de Llaves y KEMs ===

Interpretación de columnas:
  - Operación Cliente: Para ECDH clásico, es la operación ECDH del cliente.
                        Para KEMs PQC, es la Encapsulación (cliente genera ciphertext).
  - Operación Servidor: Para ECDH clásico, es la operación ECDH del servidor (idéntica).
                         Para KEMs PQC, es Keygen + Desencapsulación (servidor genera keypair y extrae secreto).
  - Tiempo Total: Suma de todas las operaciones = tiempo total del intercambio de llaves completo.

Nota: En ECDH, ambas partes ejecutan la misma operación. En KEM, el servidor debe generar claves primero.


,Algoritmo,Categoría,Operación Cliente (ms),Operación Servidor (ms),Tiempo Total (ms)
0,Classic-McEliece-460896,KEM,0.0651 ± 0.0059,118.8591 ± 7.0665,118.9242 ± 7.0665
1,ML-KEM-768,KEM,0.0375 ± 0.0032,0.0670 ± 0.0024,0.1046 ± 0.0039
2,P-256,ECDH,0.0476 ± 0.0004,0.0476 ± 0.0004,0.0952 ± 0.0009
3,X25519,ECDH,0.0322 ± 0.0015,0.0322 ± 0.0015,0.0643 ± 0.0029



=== TABLA 2B: Rendimiento de Firmas Digitales ===
* Tiempo Total = Firma + Verificación


,Algoritmo,Categoría,Firma (ms),Verificación (ms),Tiempo Total (ms)
0,ECDSA-P256,Firma Digital,0.0284 ± 0.0011,0.0631 ± 0.0021,0.0915 ± 0.0024
1,Ed25519,Firma Digital,0.0406 ± 0.0019,0.1102 ± 0.0071,0.1508 ± 0.0073
2,ML-DSA-65,Firma Digital,0.1434 ± 0.0046,0.0654 ± 0.0031,0.2088 ± 0.0056
3,RSA-2048,Firma Digital,0.6079 ± 0.0182,0.0171 ± 0.0006,0.6250 ± 0.0182


## 4. Export to CSV

In [ ]:
output_path_2a = '../bench/algorithm_metrics/table_2a_kems_performance.csv'
output_path_2b = '../bench/algorithm_metrics/table_2b_signatures_performance.csv'

table_2a.to_csv(output_path_2a, index=False, encoding='utf-8')
table_2b.to_csv(output_path_2b, index=False, encoding='utf-8')

print(f"\nTable 2A exported to: {output_path_2a}")
print(f"Table 2B exported to: {output_path_2b}")


Table 2A (KEMs) exported to: ../bench/table_2a_kems_performance.csv
Table 2B (Signatures) exported to: ../bench/table_2b_signatures_performance.csv
